In [3]:
import pandas as pd
import requests

# --- 0. Load CSV ---
url = "les-arbres.csv"
df = pd.read_csv(url, sep=';', low_memory=False)

print("Nombre d'arbres importés :", len(df))
print(df.columns)
print(df.head())

# --- 1. Convert height to numeric ---
df["HAUTEUR (m)"] = pd.to_numeric(df["HAUTEUR (m)"], errors="coerce")

print("Nombre total d'arbres :", len(df))
print("Nombre d'arbres avec hauteur renseignée :", df["HAUTEUR (m)"].notna().sum())

# --- 2. Filter by height (2m ≤ H ≤ 35m) ---
df = df[(df["HAUTEUR (m)"] >= 2) & (df["HAUTEUR (m)"] <= 35)]
print("Après filtre hauteur :", len(df), "arbres restants")

# --- 3. Keep only genera with >= 1000 trees ---
genre_counts = df["GENRE"].value_counts()
valid_genres = genre_counts[genre_counts >= 1000].index
df = df[df["GENRE"].isin(valid_genres)]
print("Après filtre genres >= 1000 arbres :", len(df), "arbres restants")
print("Genres restants :", len(valid_genres))

# Séparer latitude et longitude
df[['LAT', 'LON']] = df['geo_point_2d'].str.split(',', expand=True).astype(float)

# Garde uniquement les arbres dans le rectangle de Paris intra-muros
df = df[
    (df['LAT'] >= 48.815) & (df['LAT'] <= 48.902) &
    (df['LON'] >= 2.224) & (df['LON'] <= 2.424)
]

print("Après filtre Paris intra-muros via geo_point_2d :", len(df), "arbres restants")

# --- 4. Count distinct species / types ---
species_counts = df['ESPECE'].value_counts()
print("Number of distinct species:", species_counts.shape[0])
print("Top species:", species_counts.head(10))

# --- 5. Map genus to 5 broad categories ---
mapping = {
    # Tall / shade trees
    'Platanus': 'Arbres hauts / ombrage',
    'Tilia': 'Arbres hauts / ombrage',
    'Quercus': 'Arbres hauts / ombrage',
    'Fraxinus': 'Arbres hauts / ombrage',
    'Populus': 'Arbres hauts / ombrage',
    'Ulmus': 'Arbres hauts / ombrage',
    'Carpinus': 'Arbres hauts / ombrage',
    'Celtis': 'Arbres hauts / ombrage',
    'Fagus': 'Arbres hauts / ombrage',

    # Ornamental / decorative trees
    'Acer': 'Arbres décoratifs',
    'Styphnolobium': 'Arbres décoratifs',
    'Magnolia': 'Arbres décoratifs',
    'Paulownia': 'Arbres décoratifs',
    'Cercis': 'Arbres décoratifs',
    'Chamaecyparis': 'Arbres décoratifs',
    'Ginkgo': 'Arbres décoratifs',
    'Ostrya': 'Arbres décoratifs',

    # Fruit trees
    'Prunus': 'Fruitier',
    'Pyrus': 'Fruitier',
    'Malus': 'Fruitier',
    'Juglans': 'Fruitier',

    # Conifers / evergreens
    'Pinus': 'Conifères',
    'Taxus': 'Conifères',

    # Nitrogen-fixers / tough street trees
    'Aesculus': 'Autres',
    'Robinia': 'Autres',
    'Gleditsia': 'Autres',
    'Corylus': 'Autres'
}

# Apply mapping with fallback to "Autres"
df['CATEGORY'] = df['GENRE'].apply(lambda g: mapping.get(g, 'Autres'))

# --- 6. Count trees per category ---
cat_counts = df['CATEGORY'].value_counts()
print("\nNombre d'arbres par catégorie :")
print(cat_counts)

# --- 7. Count how many genera are in each category ---
genus_to_cat = {g: mapping.get(g, 'Autres') for g in df['GENRE'].unique()}
genus_df = pd.DataFrame(list(genus_to_cat.items()), columns=['GENRE', 'CATEGORY'])
print("\nNombre de genres par catégorie :")
print(genus_df['CATEGORY'].value_counts())


Nombre d'arbres importés : 214664
Index(['IDBASE', 'TYPE EMPLACEMENT', 'DOMANIALITE', 'ARRONDISSEMENT',
       'COMPLEMENT ADRESSE', 'LIEU / ADRESSE', 'IDEMPLACEMENT',
       'LIBELLE FRANCAIS', 'GENRE', 'ESPECE', 'VARIETE OUCULTIVAR',
       'CIRCONFERENCE (cm)', 'HAUTEUR (m)', 'STADE DE DEVELOPPEMENT',
       'REMARQUABLE', 'geo_point_2d'],
      dtype='object')
    IDBASE TYPE EMPLACEMENT DOMANIALITE     ARRONDISSEMENT COMPLEMENT ADRESSE  \
0  2011601            Arbre      Jardin  BOIS DE VINCENNES                NaN   
1  2016099            Arbre      Jardin  BOIS DE VINCENNES                NaN   
2  2034323            Arbre       DASCO    PARIS 14E ARRDT                NaN   
3  2038045            Arbre      Jardin    PARIS 20E ARRDT                NaN   
4  2047247            Arbre      Jardin    PARIS 12E ARRDT                NaN   

                                  LIEU / ADRESSE IDEMPLACEMENT  \
0    PARC FLORAL DE PARIS / ROUTE DE LA PYRAMIDE      00180090   
1    PARC FLOR

In [8]:
import re

def clean_arrdt(value):
    if pd.isna(value):
        return None
    match = re.search(r"\d+", str(value))
    return int(match.group()) if match else None

df["ARRONDISSEMENT"] = df["ARRONDISSEMENT"].apply(clean_arrdt)

# Garder uniquement les colonnes utiles
df_small = df[["ARRONDISSEMENT", "geo_point_2d", "CATEGORY"]]

# Renommer les colonnes pour plus de clarté
df_small = df_small.rename(columns={
    "ARRONDISSEMENT": "Arrondissement",
    "geo_point_2d": "Coordonnée 2D",
    "TYPE ARBRE": "Type d'arbre"
})

# 7. Exporter le nouveau CSV
df_small.to_csv("arbres_paris_centre_cleaned.csv", sep=";", index=False, encoding="utf-8")

print("✅ Nouveau CSV généré : arbres_paris_centre_cleaned.csv")
print("📌 Nombre d’arbres conservés :", len(df_small))

✅ Nouveau CSV généré : arbres_paris_centre_cleaned.csv
📌 Nombre d’arbres conservés : 153456
